# How to store revisions and reports

In this notebook we render to HTML notebooks downloaded to our Leo instance and store them to dated subdirectories in our workspace bucket so that we can have revisions.

See also [Notebooks 101 - How not to lose data output files or collaborator edits](https://broadinstitute.zendesk.com/hc/en-us/articles/360027300571-Notebooks-101-How-not-to-lose-data-output-files-or-collaborator-edits).

# Setup

First, be sure to run notebook **`R environment setup`** in this workspace.

In [1]:
library(lubridate)
library(tidyverse)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0       ✔ purrr   0.3.2  
✔ tibble  2.1.1       ✔ dplyr   0.8.0.1
✔ tidyr   0.8.3       ✔ stringr 1.4.0  
✔ readr   1.3.1       ✔ forcats 0.4.0  
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()


Get the Cloud Storage bucket associated with this workspace.

In [2]:
(WORKSPACE_BUCKET <- Sys.getenv('WORKSPACE_BUCKET'))

[1] "gs://fc-9b460d40-2985-49b5-a228-cdcf639e8c40"

Since we cannot programmatically get a username that would look decent in a URL instead use a stamp with resolution in seconds to hopefully avoid collisions between multiple users creating revisions at the same time.

In [3]:
# Create a timestamp for folder of results generated today.
(TIMESTAMP <- strftime(now(), '%Y%m%d/%H%M%S'))

[1] "20190429/211030"

In [4]:
(REPORT_DESTINATION <- str_glue('{WORKSPACE_BUCKET}/reports/{TIMESTAMP}/'))

gs://fc-9b460d40-2985-49b5-a228-cdcf639e8c40/reports/20190429/211030/

In [5]:
(PET_USER <- system('gcloud auth list --filter=status:ACTIVE --format="value(account)"', intern = TRUE))

[1] "pet-110502112294304402890@fc-product-demo.iam.gserviceaccount.com"

# Permissions check

Check that you can write to the workspace bucket.

This also serves as a way to figure out who wrote the files to the subdirectories.

In [6]:
system(str_glue(
    'echo {PET_USER} about to save copies of work from {TIMESTAMP} | gsutil cp - {REPORT_DESTINATION}status.txt'),
       intern = TRUE)

character(0)

In [7]:
system(str_glue('gsutil cat {REPORT_DESTINATION}status.txt'), intern = TRUE)

[1] "pet-110502112294304402890@fc-product-demo.iam.gserviceaccount.com about to save copies of work from 20190429/211030"

# Save copies

This notebook is currently written to only create revisions for notebooks that have been delocalized to this particular Leonardo VM. (e.g., only the notebooks I worked on recently).

In [8]:
list.files()

[1] "1"                                                                 
[2] "R - How to save and load R objects from the workspace bucket.ipynb"
[3] "R - How to store revisions and reports.ipynb"                      
[4] "thousand_genomes_20190429-210345.rda"                              
[5] "thousand_genomes_20190429-210550.rda"                              
[6] "thousand_genomes_20190429-210705.rda"

## Render notebooks to HTML

**Note**: this does not re-run them, it merely takes the currently autosaved version and converts it to HTML.

In [9]:
map(list.files(pattern = '*.ipynb'),
   function(nb) {
       print(system(str_glue('jupyter nbconvert --to html \'{nb}\''), intern = TRUE))
   })

character(0)
character(0)


[[1]]
character(0)

[[2]]
character(0)

In [10]:
list.files(pattern = '*.html')

[1] "R - How to save and load R objects from the workspace bucket.html"
[2] "R - How to store revisions and reports.html"

In [11]:
system(str_glue('gsutil -m cp *.html {REPORT_DESTINATION}'), intern = TRUE)

character(0)

# Provenance

In [12]:
devtools::session_info()

─ Session info ───────────────────────────────────────────────────────────────
 setting  value                       
 version  R version 3.5.2 (2018-12-20)
 os       Debian GNU/Linux 9 (stretch)
 system   x86_64, linux-gnu           
 ui       X11                         
 language (EN)                        
 collate  en_US.UTF-8                 
 ctype    en_US.UTF-8                 
 tz       Etc/UTC                     
 date     2019-04-29                  

─ Packages ───────────────────────────────────────────────────────────────────
 package     * version     date       lib source                            
 assertthat    0.2.1       2019-03-21 [2] CRAN (R 3.5.2)                    
 backports     1.1.3       2018-12-14 [2] CRAN (R 3.5.2)                    
 base64enc     0.1-3       2015-07-28 [2] CRAN (R 3.5.2)                    
 broom         0.5.1       2018-12-05 [2] CRAN (R 3.5.2)                    
 callr         3.2.0       2019-03-15 [2] CRAN (R 3.5.2)          

Copyright 2019 The Broad Institute, Inc., Verily Life Sciences, LLC
All rights reserved.

This software may be modified and distributed under the terms of the BSD license.  See the LICENSE file for details.